# Limpieza de datos sobre edificios con certificacion LEED
## 1. Introduccion

EL United States Green Building Council (USGBG) tiene una base de datos de edificios que cuentan con certificación LEED alrededor del mundo. La pagina web de USGBG cuenta con una interfaz para hacer consultas directamente a su base de datos, sin embargo no cuenta con una API o una URL directa para descarga masiva por lo que es necesario enviar el query a la base de datos desde la URL de USBG:
https://www.usgbc.org/projects/list?page=17&keys=Mexico

Después de esperar a que la base de datos interprete el query, regresa el archivo *"leed_projects.xls"* que quedará guardado como *"D:\PCCS\00_RawData\01_CSV\LEED\leed_projects.xls"*

## 2. Estandarizacion del dataset

In [2]:
# Librerias utilizadas
import pandas as pd
import sys
import os
import csv
from lxml import html
import requests
import time

In [2]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


In [3]:
# Importar datos (El dataset no se puede importar directamente como descargado porque envia un error)
path = r'D:\PCCS\00_RawData\01_CSV\LEED\leed_projects.xls'
raw_data = pd.read_excel(path)
raw_data.head()

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'<html>\n '

El archivo tal como se descarga, a pesar de ser tabulados de excel, envia un mensaje de error cuando se intenta abrir directamente como se descargó. Por lo tanto, antes de procesarlo es necesario abrirlo en excel y guardarlo con formato .xlsx

In [4]:
path = r'D:\PCCS\00_RawData\01_CSV\LEED\leed_projects.xlsx'
raw_data = pd.read_excel(path)
raw_data.index.name = 'Building'
raw_data.head()

,Path,Certification date,City,State,Country,Rating system,Version,Certification level
Building,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver
Talleres de Innovacion para el Diseno,https://www.usgbc.org/projects/talleres-de-inn...,2012-06-15,Tlaquepaque,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver
Fortius casa Hidalgo,https://www.usgbc.org/projects/fortius-casa-hi...,2014-01-07,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Platinum


In [5]:
# Eliminar columnas que no pertenecen a Mexico

print('La tabla tiene {} registros'.format(len(raw_data)))
x = 'United States [us]'
raw_data = raw_data[raw_data['Country'] != x]
print('Quitando los registros donde el país es "{}", la tabla queda con {} registros'.format(x, len(raw_data)))
x = 'Colombia'
raw_data = raw_data[raw_data['Country'] != x]
print('Quitando los registros donde el país es "{}", la tabla queda con {} registros'.format(x, len(raw_data)))
x = 'United States'
raw_data = raw_data[raw_data['Country'] != x]
print('Quitando los registros donde el país es "{}", la tabla queda con {} registros'.format(x, len(raw_data)))

raw_data.head()

La tabla tiene 2390 registros
Quitando los registros donde el país es "United States [us]", la tabla queda con 790 registros
Quitando los registros donde el país es "Colombia", la tabla queda con 789 registros
Quitando los registros donde el país es "United States", la tabla queda con 784 registros


,Path,Certification date,City,State,Country,Rating system,Version,Certification level
Building,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver
Talleres de Innovacion para el Diseno,https://www.usgbc.org/projects/talleres-de-inn...,2012-06-15,Tlaquepaque,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver
Fortius casa Hidalgo,https://www.usgbc.org/projects/fortius-casa-hi...,2014-01-07,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Platinum


La base de datos es un listado de edificios que incluye para cada edificio: 
- El nombre del edificio
- Una URL de referencia
- La fecha de la certificacion del edificio
- La ciudad, estado y país en el que se ubica el edificio
- El sistema de calificación bajo el ecual se certificó el edificio
- La versión de la certificación
- El nivel alcanzado por el edificio con la certificación.

Debido a que las columnas de Ciudad, estado y país no están realizadas bajo ningun estándar, es necesario asignar a cada renglón las claves geoestadísticas municipales de 5 dígitos correspondientes al municipio en el que se ubica el edificio.
Esto se hará manualmente pues cada renglón tiene que ser interpretado individualmente.

Durante la revision me di cuenta que si bien la tabla no tiene una clave para identificar cada ciudad y municipio, la liga de cada edificio nos lleva a una ficha del municipio que usualmente sí contiene un código postal; y desde el código postal es posible obtener el municipio y el estado.
A continuacion se hace la revision de una pagina para conocer su estructura y hacer un webscrapping desde esta estructura, esperando que sea igual en todas las fichas:

In [6]:
# Descarga el HTML de la pagina
page = requests.get('https://www.usgbc.org/projects/reforma-180')
tree = html.fromstring(page.content)

In [7]:
# Obten variables desde la estructura
street = tree.xpath('//span[@itemprop="streetAddress"]/text()')
locality = tree.xpath('//span[@itemprop="addressLocality"]/text()')
postalcode = tree.xpath('//span[@itemprop="postalCode"]/text()')
country = tree.xpath('//span[@itemprop="addressCountry"]/text()')

In [8]:
''.join(street).replace('\n', '')

'Paseo De La Reforma 180'

In [9]:
# A ver, que datos sacaste?
print('len({}), type({}) - {}'.format(len(street), type(street), street))
print('len({}), type({}) - {}'.format(len(locality), type(locality), locality))
print('len({}), type({}) - {}'.format(len(postalcode), type(postalcode), postalcode))
print('len({}), type({}) - {}'.format(len(country), type(country), country))

len(2), type(<class 'list'>) - ['\nPaseo De La Reforma 180', '\n']
len(1), type(<class 'list'>) - ['Ciudad de Mexico']
len(1), type(<class 'list'>) - ['06600']
len(1), type(<class 'list'>) - ['Mexico']


Todos los datos son listas, pero "street" tiene 2 elementos. Entonces para el script lo que voy a hacer será eliminar todos los saltos de linea y concatenar el texto de todos los elementos de la lista

In [10]:
# Script para extraer datos de fichas a partir de la URL
def webcrawler(x):
    time.sleep(0.05)
    url = x
    try:
        page = requests.get(x)
        tree = html.fromstring(page.content)
    except:           # Regresa false si no logras entrar a la URL
        street = False
        locality = False
        postalcode = False
        country = False
        return [street, locality, postalcode, country]
    # Saca los datos del tree. Regresa None si no encontraste 
    try:
        street = ''.join(tree.xpath('//span[@itemprop="streetAddress"]/text()'))
    except:
        street = None
    try:
        locality = tree.xpath('//span[@itemprop="addressLocality"]/text()')
    except:
        locality = None
    try:
        postalcode = tree.xpath('//span[@itemprop="postalCode"]/text()')
    except:
        postalcode = None
    try:
        country = tree.xpath('//span[@itemprop="addressCountry"]/text()')
    except:
        country = None
    
    return [street, locality, postalcode, country]

In [11]:
# Pon al crawler a hacer su chamba (Pero no si el archivo ya existe)
archivoraw = r'D:\PCCS\00_RawData\01_CSV\LEED\crawl_leed.xlsx'
if os.path.isfile(archivoraw):
    print('NO SE REALIZÓ EL WEBCRAWL PORQUE YA SE TIENEN LOS DATOS EN \n {}'.format(archivoraw))
    print('*** Mejor importa el archivo para no gastar tantos recursos ***')
else:
    raw_data['crawl'] = raw_data.Path.apply(webcrawler)

Reemplaza los enters en cada lista

(Voy a saltarme este paso porque lo que me interesa en realidad es el Codigo Postal, pero dejo el codigo por si lo ocupo en el futuro)

def listtotext(x):
    templist = []
    for element in x:
        if element == None or element == False:
            templist.append(element)
        else:
            templist.append(''.join(x).replace('\n', ''))
    return templist

In [244]:
raw_data.head()

,Path,Certification date,City,State,Country,Rating system,Version,Certification level,crawl
Building,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M..."
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50..."
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPablo Neruda 2631\n, [Guadalajara], [44630]..."
Talleres de Innovacion para el Diseno,https://www.usgbc.org/projects/talleres-de-inn...,2012-06-15,Tlaquepaque,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPeriferico Sur Manuel Gomez Morin 8585\n, [..."
Fortius casa Hidalgo,https://www.usgbc.org/projects/fortius-casa-hi...,2014-01-07,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Platinum,"[\nAv. Hidalgo 1291\n, [Guadalajara], [44600],..."


In [496]:
# Guarda una copia de raw_data por si es necesario ocupar este dataset de nuevo, 
# que no se tenga que hacer nuevamente el webcrawiling porque consume mucho tiempo
writer = pd.ExcelWriter(archivoraw)
raw_data.to_excel(writer, sheet_name = 'DATOS')
writer.save()

In [470]:
# Crea una copia de trabajo de raw_data
datasetfinal = raw_data

In [471]:
# Crea una columna única con los datos de dirección y código postal extraídos con el crawler.
datasetfinal['address'] = datasetfinal.crawl.apply(lambda x: x[0].replace('\n', ''))
# raw_data['city'] = raw_data.crawl.apply(lambda x: x[1][0].replace('/n', ''))
datasetfinal['CP'] = datasetfinal.crawl.apply(lambda x: str(x[2][0]))
# raw_data['city'] = raw_data.crawl.apply(lambda x: x[3][0].replace('/n', ''))
datasetfinal.head(2)

,Path,Certification date,City,State,Country,Rating system,Version,Certification level,crawl,address,CP
Building,,,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M...",Puerto Interior S/n,36275
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50...",Km 63 Carretera Mex-Toluca,50140


A partir de los Codigos Postales ya es posible identificar la ciudad y municipio a la que pertenece cada edificio. Para esto, vamos a utilizar la base de datos de codigos postales del SEPOMEX que se descargó en otra minería de datos:

In [472]:
bd_sepo = r'D:\PCCS\01_Dmine\Datasets\SEPOMEX\sepomex_CP_CVEMUN.xlsx'
SEPOMEX = pd.read_excel(bd_sepo, dtype={'CVE_MUN':'str', 'CP':'str'})
SEPOMEX.head(3)

,CP,CVE_MUN
0,20000,01001
1,20010,01001
6,20016,01001


Con la base de datos del SEPOMEX ya es posible unir ambos datasets para obtener las claves municipales de cada edificio

In [473]:
datasetfinal.head()

,Path,Certification date,City,State,Country,Rating system,Version,Certification level,crawl,address,CP
Building,,,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M...",Puerto Interior S/n,36275
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50...",Km 63 Carretera Mex-Toluca,50140
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPablo Neruda 2631\n, [Guadalajara], [44630]...",Pablo Neruda 2631,44630
Talleres de Innovacion para el Diseno,https://www.usgbc.org/projects/talleres-de-inn...,2012-06-15,Tlaquepaque,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPeriferico Sur Manuel Gomez Morin 8585\n, [...",Periferico Sur Manuel Gomez Morin 8585,45604
Fortius casa Hidalgo,https://www.usgbc.org/projects/fortius-casa-hi...,2014-01-07,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Platinum,"[\nAv. Hidalgo 1291\n, [Guadalajara], [44600],...",Av. Hidalgo 1291,44600


In [474]:
# Copiar CVE_MUN del dataset en base al codigo postal
datasetfinal = datasetfinal.reset_index().merge(SEPOMEX, on='CP', how='left').set_index('Building')
datasetfinal.head()

,Path,Certification date,City,State,Country,Rating system,Version,Certification level,crawl,address,CP,CVE_MUN
Building,,,,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M...",Puerto Interior S/n,36275,11037
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50...",Km 63 Carretera Mex-Toluca,50140,15106
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPablo Neruda 2631\n, [Guadalajara], [44630]...",Pablo Neruda 2631,44630,14039
Talleres de Innovacion para el Diseno,https://www.usgbc.org/projects/talleres-de-inn...,2012-06-15,Tlaquepaque,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPeriferico Sur Manuel Gomez Morin 8585\n, [...",Periferico Sur Manuel Gomez Morin 8585,45604,14098
Fortius casa Hidalgo,https://www.usgbc.org/projects/fortius-casa-hi...,2014-01-07,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Platinum,"[\nAv. Hidalgo 1291\n, [Guadalajara], [44600],...",Av. Hidalgo 1291,44600,14039


Quedan 70 filas en donde no fue posible identificar la clave Municipal

In [475]:
len(datasetfinal[datasetfinal['CVE_MUN'].isnull()])

70

#### Casos Particulares
Estos 70 registros tienen 33 claves unicas de C que requieren ser asignadas individualmente para conocer la CVE_MUN de cada edificio. Para esto, haremos un script que permita revisar cada clave para realizar la investigación necesaria y asignarle una CVE_MUN

In [476]:
mira = ['City', 'State', 'CP', 'address', 'CVE_MUN']    # El diccionario 'mira' se utilizará en adelante para imprimir subsets de la informacion
sinmun = datasetfinal[datasetfinal['CVE_MUN'].isnull()][mira]
sinmun.head()

,City,State,CP,address,CVE_MUN
Building,,,,,
Planta Atizapan CM,Atizapan De Zaragoza,Mexico [MEX],05940,Ignacio Zaragoza 18,NaN
CityExpress Santa Fe,Mexico,[],01209,Juan Salvador Garza No. 69 Santa Fe,NaN
Hospital Regional Tlalnepantla ISSEMYM,TLALNEPANTLA DE BAZ,Mexico [MEX],05490,CALLE PASEO DEL FERROCARRIL ESQUINA AV. INDECO,NaN
Torre Metropoli,Mexico City,[],11010,Blvd. Manuel Avila Camacho No. 118,NaN
Tequila Hotel,"Tequila, Jalisco",[],08424,10 Bicente Albino Rojas,NaN


In [477]:
len(sinmun['CP'].unique())

33

En el siguiente diccionario recopila las CVE_MUN que se asignarán a los códigos postales que requieren asignacion individual. Los códigos cuyo valor es None se asignarán mas adelante

In [478]:
# Diccionario creado en donde key = 'CP' y value = 'CVE_MUN'
defmuns = {'00000': None,
 '00100': '09010',
 '00502': '15024',
 '00604': '15121',
 '00702': '15051',
 '01006': '09010',
 '01152': '09010',
 '01209': '09004',
 '01300': '09004',
 '03130': '09014',
 '03210': '09014',
 '05300': '09004',
 '05490': '15104',
 '05940': '15013',
 '08424': '14094',
 '11010': '09016',
 '11111': '14098',
 '11570': '09016',
 '12345': None,
 '21118': '02002',
 '22320': '02004',
 '23410': '03008',
 '23479': '03008',
 '31240': '08019',
 '46685': '14006',
 '48219': '16053',
 '56277': '15099',
 '66601': '19006',
 '67114': '19026',
 '76232': '22014',
 '77780': '23009',
 '78341': '24028',
 '87131': None}

El siguiente diccionario incluye códigos postales que requieren ser corregidos

In [479]:
# Diccionario en donde key = Codigo postal listado en el dataset; value = Codigo postal correcto
deberiaser = {'00100': '45620',
 '00502': '54830',
 '00604': '54713',
 '00702': '52004',
 '03130': '03103',
 '11111': '45620',
 '48219': '58218'}

#### Asignacion de codigos postales

In [480]:
# Reemplazar las CVE_MUN identificadas en el dataset final
datasetfinal['CVE_MUN'] = datasetfinal['CP'].map(defmuns).fillna(datasetfinal['CVE_MUN'])

Algunos edificios, marcados con los codigos postales 00000 y 12345 (Intuyo que por desidia del capturista) se tendrán que asignar individualmente

In [481]:
sinmun.loc[sinmun['CP'].isin(['00000', '12345'])]

,City,State,CP,address,CVE_MUN
Building,,,,,
Grainger Mexico HQ,Monterrey,[],00000,TBD,NaN
La Concha Pearl,La Paz BCS,[],00000,Highway Pichilingue,NaN
Schneider Electric at COK,Apodaca,[],00000,VYa De La Innovaci_n #408,NaN
Bank of America-Reforma 115 5th floor,Mexico DF,[],00000,Paseo De La Reforma 115,NaN
Vesta Corporate Headquarters,Mexico City,[],00000,"Piso 28, Torres 28 Paseo de Los Tamarindos 400-A",NaN
Air Traffic Control Tower,Mexico City,[],00000,TBD,NaN
Eurocenter 2,NaN,NaN,12345,"Av Juan Salvador Agraz 61, CEDEC, Santa Fe",NaN
ROUZ TOWER,MEXICO CITY,[],12345,AV. INSURGENTES 838,NaN
Passenger Terminal Building,Mexico City,[],00000,TBD,NaN


In [482]:
# Diccionario con edificios que se asignaran individualmente
# Para este diccionario key = Nombre del edificio, value =  CVE_MUN que se asignará a este edificio
buildings = {
    'Grainger Mexico HQ': '19039',
    'La Concha Pearl': '03003',
    #'Schneider Electric at COK': '66629',   # Este edificio esta repetido, por lo que no se le asignará nada y se eliminará al final
    'Bank of America-Reforma 115 5th floor': '09016',
    'Vesta Corporate Headquarters': '09016',
    'Air Traffic Control Tower': '15101',   # Estoy considerando que esta es la Torre de Control del NAICM
    'Passenger Terminal Building': '15101',    # El edificio del NAICM
    'Area Control Center': '15101',    # Infraestructura del NAICM
    'Corporativo TRIO': '09004',
    'Casa GF': '19019',
    'Eurocenter 2': '09004',
    'ROUZ TOWER': '09014',
    'Periferico Sur Parque Industrial': '14098'
}


In [483]:
# Hay un edificio duplicado. El duplicado se eliminará mas adelante
datasetfinal.loc['Schneider Electric at COK'][mira]

,City,State,CP,address,CVE_MUN
Building,,,,,
Schneider Electric at COK,Apodaca,[],00000,VYa De La Innovaci_n #408,NaN
Schneider Electric at COK,Apodaca,[],66629,VYa De La Innovaci_n #408,19006


In [484]:
# Reemplazar valores individuales en el dataset.
for k, v in buildings.items():
    building = datasetfinal.loc[k].name
    CVEMUN_prev = datasetfinal.loc[k]['CVE_MUN']
    datasetfinal.at[k, 'CVE_MUN'] = v
    print('Edificio:{} - la CVE_MUN {} se reemplazó por {}'.format(building, CVEMUN_prev, datasetfinal.at[k, 'CVE_MUN']))

Edificio:Grainger Mexico HQ - la CVE_MUN nan se reemplazó por 19039
Edificio:La Concha Pearl - la CVE_MUN nan se reemplazó por 03003
Edificio:Bank of America-Reforma 115 5th floor - la CVE_MUN nan se reemplazó por 09016
Edificio:Vesta Corporate Headquarters - la CVE_MUN nan se reemplazó por 09016
Edificio:Air Traffic Control Tower - la CVE_MUN nan se reemplazó por 15101
Edificio:Passenger Terminal Building - la CVE_MUN nan se reemplazó por 15101
Edificio:Area Control Center - la CVE_MUN nan se reemplazó por 15101
Edificio:Corporativo TRIO - la CVE_MUN nan se reemplazó por 09004
Edificio:Casa GF - la CVE_MUN nan se reemplazó por 19019
Edificio:Eurocenter 2 - la CVE_MUN nan se reemplazó por 09004
Edificio:ROUZ TOWER - la CVE_MUN nan se reemplazó por 09014
Edificio:Periferico Sur Parque Industrial - la CVE_MUN nan se reemplazó por 14098


El dataset contiene dos edificios en el dataset que no corresponden a México:

In [485]:
sinmun[sinmun['CP'] == '87131']

,City,State,CP,address,CVE_MUN
Building,,,,,
Physics & Astronomy Interdisciplinary Sc,NaN,NaN,87131,University of New Mexico,NaN
UNM Johnson Gym Addition,NaN,NaN,87131,1 University of New Mexico,NaN


Se eliminarán del dataset los siguientes edificios:

In [486]:
datasetfinal[datasetfinal['CVE_MUN'].isnull()][mira]

,City,State,CP,address,CVE_MUN
Building,,,,,
Schneider Electric at COK,Apodaca,[],00000,VYa De La Innovaci_n #408,NaN
Physics & Astronomy Interdisciplinary Sc,NaN,NaN,87131,University of New Mexico,NaN
UNM Johnson Gym Addition,NaN,NaN,87131,1 University of New Mexico,NaN


El primero por estar repetido y el resto por que no están en los Estados Unidos Mexicanos.

In [487]:
datasetfinal = datasetfinal.dropna(subset=['CVE_MUN'])
datasetfinal.head(3)

,Path,Certification date,City,State,Country,Rating system,Version,Certification level,crawl,address,CP,CVE_MUN
Building,,,,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M...",Puerto Interior S/n,36275,11037
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50...",Km 63 Carretera Mex-Toluca,50140,15106
Corporativo Atmosfera,https://www.usgbc.org/projects/corporativo-atm...,2012-04-26,Guadalajara,Jalisco [JAL],Mexico [mx],New Construction,v2009,Silver,"[\nPablo Neruda 2631\n, [Guadalajara], [44630]...",Pablo Neruda 2631,44630,14039


Los edificios que requieren correccion de codigos postales son los siguientes:

In [488]:
datasetfinal[datasetfinal['CP'].isin(list(deberiaser.keys()))][mira]

,City,State,CP,address,CVE_MUN
Building,,,,,
in high park,MUxico,[],03130,Insurgentes 430,09014
"ProLogis Park Elsato, Bldg #12",Jalisco,[],11111,Mariano Otero,14098
"ProLogis Park Elsalto, Bldg #13",Jalisco,[],11111,Carretera A San Martin,14098
"ProLogis Park Elsalto, Bldg #13",Jalisco,[],11111,Carretera A San Martin,14098
"ProLogis Park Elsato, Bldg #12",Jalisco,[],11111,Mariano Otero,14098
Corporativo Alfonso Caso,NaN,NaN,00100,Calle Alfonso Caso 106,09010
Prologis Park Laurel Bldg 2,Cuautitlan,Mexico [MEX],00502,Rio Neva S/N,15024
Prologis Park Toluca Bldg 2,ExHacienda Doria Rosa,Mexico [MEX],00702,Calle Emiliano,15051
Prologis Park Izcalli Bldg 4,NaN,NaN,00604,Carretera Mexico,15121


In [489]:
# Corregir codigos postales erróneos
datasetfinal['CP'] = datasetfinal['CP'].map(deberiaser).fillna(datasetfinal['CP'])
datasetfinal[mira].head()

,City,State,CP,address,CVE_MUN
Building,,,,,
Agregados de Guanajuato Planta Motores,NaN,NaN,36275,Puerto Interior S/n,11037
Pfizer Administrative Building Toluca,Toluca,Mexico [MEX],50140,Km 63 Carretera Mex-Toluca,15106
Corporativo Atmosfera,Guadalajara,Jalisco [JAL],44630,Pablo Neruda 2631,14039
Talleres de Innovacion para el Diseno,Tlaquepaque,Jalisco [JAL],45604,Periferico Sur Manuel Gomez Morin 8585,14098
Fortius casa Hidalgo,Guadalajara,Jalisco [JAL],44600,Av. Hidalgo 1291,14039


In [490]:
# Renombrar columnas para crear variables únicas
columns={
    'address':'direccion',
    'Path': 'URL',
    'Certification date': 'usgbc_fecha_cert', 
    'Rating system':'usgbc_sis_val',
    'Version': 'usgbc_ver_sisv',
    'Certification level': 'usgbc_nv_cert',
}
datasetfinal = datasetfinal.rename(columns=columns)
datasetfinal.head(2)

,URL,usgbc_fecha_cert,City,State,Country,usgbc_sis_val,usgbc_ver_sisv,usgbc_nv_cert,crawl,direccion,CP,CVE_MUN
Building,,,,,,,,,,,,
Agregados de Guanajuato Planta Motores,https://www.usgbc.org/projects/agregados-de-gu...,2015-05-12,NaN,NaN,Mexico,New Construction,v2009,Gold,"[\nPuerto Interior S/n\n, [Silao], [36275], [M...",Puerto Interior S/n,36275,11037
Pfizer Administrative Building Toluca,https://www.usgbc.org/projects/pfizer-administ...,2014-05-12,Toluca,Mexico [MEX],Mexico [mx],New Construction,v2009,Gold,"[\nKm 63 Carretera Mex-Toluca\n, [Toluca], [50...",Km 63 Carretera Mex-Toluca,50140,15106


In [491]:
# Descripciones de columnas
variables = {
    'direccion': 'Ubicacion (Calle y numero)',
    'CVE_MUN': 'Clave geoestadística de 5 digitos a nivel municipal, de acuerdo con el Catálogo Único de Claves de Áreas Geoestadísticas Estatales, Municipales y Localidades de INEGI',
    'usgbc_fecha_cert': 'Fecha de certificacion como edificio LEED por el United States Green Building Council',
    'usgbc_sis_val': 'Sistema de valoracion aplicado por el United States Green Building Council al edificio',
    'usgbc_ver_sisv': 'Version del Sistema de valoracion aplicado por el United States Green Building Council al edificio',
    'usgbc_nv_cert': 'Nivel de certificacion como edificio LEED alcanzado por el edificio',
    'CP': 'Codigo Postal',
    'URL': 'Uniform Resource Locator, referencia a recurso en línea'
}

# Convertir descripciones a dataframe
variables = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
variables.columns = ['Descripcion']
variables = variables.rename_axis('Mnemonico')
variables.head()

,Descripcion
Mnemonico,
direccion,Ubicacion (Calle y numero)
CVE_MUN,Clave geoestadística de 5 digitos a nivel muni...
usgbc_fecha_cert,Fecha de certificacion como edificio LEED por ...
usgbc_sis_val,Sistema de valoracion aplicado por el United S...
usgbc_ver_sisv,Version del Sistema de valoracion aplicado por...


In [492]:
# Eliminar columnas que ya no se utilizarán y reordenar
setfinal = [
 'direccion',
 'CVE_MUN',
 'usgbc_fecha_cert',
 'usgbc_sis_val',
 'usgbc_ver_sisv',
 'usgbc_nv_cert',
 'CP',
 'URL']
datasetfinal = datasetfinal[setfinal]
datasetfinal.head()

,direccion,CVE_MUN,usgbc_fecha_cert,usgbc_sis_val,usgbc_ver_sisv,usgbc_nv_cert,CP,URL
Building,,,,,,,,
Agregados de Guanajuato Planta Motores,Puerto Interior S/n,11037,2015-05-12,New Construction,v2009,Gold,36275,https://www.usgbc.org/projects/agregados-de-gu...
Pfizer Administrative Building Toluca,Km 63 Carretera Mex-Toluca,15106,2014-05-12,New Construction,v2009,Gold,50140,https://www.usgbc.org/projects/pfizer-administ...
Corporativo Atmosfera,Pablo Neruda 2631,14039,2012-04-26,New Construction,v2009,Silver,44630,https://www.usgbc.org/projects/corporativo-atm...
Talleres de Innovacion para el Diseno,Periferico Sur Manuel Gomez Morin 8585,14098,2012-06-15,New Construction,v2009,Silver,45604,https://www.usgbc.org/projects/talleres-de-inn...
Fortius casa Hidalgo,Av. Hidalgo 1291,14039,2014-01-07,New Construction,v2009,Platinum,44600,https://www.usgbc.org/projects/fortius-casa-hi...


In [496]:
metadatos = {
    'Nombre del Dataset': 'Edificios con Certificacion LEED',
    'Descripcion del dataset': 'Edificios que han recibido algún nivel de certificación de Liderazgo en Energía y desarrollo Ambiental' \
              ' (LEED, por sus siglas en ingles) Otorgado por el Consejo de edificios Verdes de Estados Unidos (USGBC' \
              ' por sus suglas en inglés)',
    'Disponibilidad Temporal': '2007 - 2018',
    'Periodo de actualizacion': 'No Definido',
    'Nivel de Desagregacion': 'Edificio',
    'Notas': 's/n',
    'Fuente': 'United States Green Buildings Council',
    'URL_Fuente': 'https://www.usgbc.org/projects/list?page=17&keys=Mexico',
    'Dataset base': None
}

# Metadatos a dataframe para exportar
metadatos = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
metadatos.columns = ['Descripcion']
metadatos = metadatos.rename_axis('Metadato')
metadatos

,Descripcion
Metadato,
Nombre del Dataset,Edificios con Certificacion LEED
Descripcion del dataset,Edificios que han recibido algún nivel de cert...
Disponibilidad Temporal,2007 - 2018
Periodo de actualizacion,No Definido
Nivel de Desagregacion,Edificio
Notas,s/n
Fuente,United States Green Buildings Council
URL_Fuente,https://www.usgbc.org/projects/list?page=17&ke...
Dataset base,None


In [497]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\LEED\PCCS_leed_projects.xlsx'
writer = pd.ExcelWriter(file)
datasetfinal.to_excel(writer, sheet_name = 'DATOS')
metadatos.to_excel(writer, sheet_name = 'METADATOS')
variables.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------
